# Shot Chart Exploration

In [3]:
import nba_api.stats.endpoints as nba
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [4]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

## Retrieve Shot Charts for Each Player in Dataset and Upload to Database

### Function to get shot chart for player and team combination

In [49]:
def get_shot_chart(team_id, player_id, date_from='', date_to='', context_measure_simple='FGA'):
    return nba.ShotChartDetail(team_id=team_id,player_id=player_id, date_from_nullable=date_from, date_to_nullable=date_to, context_measure_simple='FGA',timeout=60).get_data_frames()[0]

### Generate list of team and player ids

In [42]:
stmt = "SELECT TEAM_ID, PLAYER_ID FROM box_score"
cursor.execute(stmt)
team_player_combos = cursor.fetchall()

In [43]:
unique = list(set(team_player_combos))

### Get field names from shot_chart table to build insert statement

In [44]:
stmt = "SELECT * from nba.shot_chart"
cursor.execute(stmt)
scores = cursor.fetchall()

field_names = [i[0] for i in cursor.description]

In [47]:
insert_statement = "INSERT INTO nba.shot_chart (GRID_TYPE, GAME_ID, GAME_EVENT_ID, PLAYER_ID, PLAYER_NAME, TEAM_ID, TEAM_NAME, PERIOD, MINUTES_REMAINING, SECONDS_REMAINING, EVENT_TYPE, ACTION_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, SHOT_ZONE_RANGE, SHOT_DISTANCE, LOC_X, LOC_Y, SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG, GAME_DATE, HTM, VTM) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

### Run function and insert into database

In [51]:
count = 0
for combo in unique:
    df = get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' )
    df_reordered = df[field_names]
    stats = [tuple(row) for row in df_reordered.values]
    cursor.executemany(insert_statement, stats)
    cnx.commit()
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Re-run function starting at timeout index from above

In [53]:
count = 0
for combo in unique[811:]:
    df = get_shot_chart(combo[0], combo[1], date_from ='10-17-2017', date_to = '11-24-2019' )
    df_reordered = df[field_names]
    stats = [tuple(row) for row in df_reordered.values]
    cursor.executemany(insert_statement, stats)
    cnx.commit()
    count+=1
    print (count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


## Explore Shot Charts for Preparation for Clustering

In [11]:
stmt = "SELECT * from nba.shot_chart"
cursor.execute(stmt)
scores = cursor.fetchall()

field_names = [i[0] for i in cursor.description]
shots = pd.DataFrame(scores)
shots.columns = field_names

In [12]:
shots.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [13]:
shots.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021700885,643,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,31,...,Center(C),Less Than 8 ft.,2,-25,5,1,0,20180224,MIA,MEM
1,Shot Chart Detail,0021700885,645,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,27,...,Center(C),Less Than 8 ft.,0,0,-6,1,0,20180224,MIA,MEM
2,Shot Chart Detail,0021700898,657,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,1,19,...,Center(C),Less Than 8 ft.,2,17,20,1,1,20180226,BOS,MEM
3,Shot Chart Detail,0021700898,671,1627744,Brice Johnson,1610612763,Memphis Grizzlies,4,0,12,...,Center(C),Less Than 8 ft.,2,-18,10,1,1,20180226,BOS,MEM
4,Shot Chart Detail,0021700931,124,1627744,Brice Johnson,1610612763,Memphis Grizzlies,1,2,36,...,Center(C),8-16 ft.,15,23,151,1,0,20180302,MEM,DEN


### Analysis of Action Types

In [6]:
shots.ACTION_TYPE.unique()

array(['Layup Shot', 'Tip Layup Shot', 'Driving Layup Shot',
       'Cutting Dunk Shot', 'Jump Shot', 'Cutting Layup Shot',
       'Hook Shot', 'Floating Jump shot', 'Putback Layup Shot',
       'Cutting Finger Roll Layup Shot', 'Dunk Shot',
       'Driving Floating Jump Shot', 'Driving Finger Roll Layup Shot',
       'Running Jump Shot', 'Turnaround Jump Shot', 'Running Layup Shot',
       'Driving Dunk Shot', 'Pullup Jump shot', 'Driving Hook Shot',
       'Reverse Layup Shot', 'Running Dunk Shot',
       'Turnaround Bank Hook Shot', 'Driving Reverse Layup Shot',
       'Turnaround Fadeaway shot', 'Fadeaway Jump Shot',
       'Running Finger Roll Layup Shot', 'Finger Roll Layup Shot',
       'Alley Oop Dunk Shot', 'Step Back Jump shot',
       'Running Pull-Up Jump Shot', 'Jump Bank Shot',
       'Alley Oop Layup shot', 'Driving Floating Bank Jump Shot',
       'Running Reverse Layup Shot', 'Turnaround Hook Shot',
       'Tip Dunk Shot', 'Putback Dunk Shot',
       'Turnaround Fadeaw

In [7]:
len(shots.ACTION_TYPE.unique())

46

### Filter Dataframe for 2017 season only

In [2]:
shots17 = shots[shots.GAME_ID.str.contains('00217')]

NameError: name 'shots' is not defined

### Convert action_type columns into dummy variables so that I can apply aggregation functions to these to see the percentage of field goal attempts each player takes by shot type

In [13]:
shots17_dummy = pd.get_dummies(shots17, columns=['ACTION_TYPE','SHOT_TYPE'], prefix=['',''], prefix_sep=['',''])

In [14]:
shots17_dummy.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM',
       'Alley Oop Dunk Shot', 'Alley Oop Layup shot', 'Cutting Dunk Shot',
       'Cutting Finger Roll Layup Shot', 'Cutting Layup Shot',
       'Driving Bank Hook Shot', 'Driving Dunk Shot',
       'Driving Finger Roll Layup Shot', 'Driving Floating Bank Jump Shot',
       'Driving Floating Jump Shot', 'Driving Hook Shot', 'Driving Layup Shot',
       'Driving Reverse Dunk Shot', 'Driving Reverse Layup Shot', 'Dunk Shot',
       'Fadeaway Jump Shot', 'Finger Roll Layup Shot', 'Floating Jump shot',
       'Hook Bank Shot', 'Hook Shot', 'Jump Bank Shot', 'Jump Shot',
       'Layup Shot', 'Pullup Jump shot', 'Putback Dunk Shot',
       'Putb

In [15]:
keep_cols =['PLAYER_ID', 'PLAYER_NAME', 'SHOT_DISTANCE', 'SHOT_MADE_FLAG','Alley Oop Dunk Shot', 'Alley Oop Layup shot', 'Cutting Dunk Shot',
       'Cutting Finger Roll Layup Shot', 'Cutting Layup Shot',
       'Driving Bank Hook Shot', 'Driving Dunk Shot',
       'Driving Finger Roll Layup Shot', 'Driving Floating Bank Jump Shot',
       'Driving Floating Jump Shot', 'Driving Hook Shot', 'Driving Layup Shot',
       'Driving Reverse Dunk Shot', 'Driving Reverse Layup Shot', 'Dunk Shot',
       'Fadeaway Jump Shot', 'Finger Roll Layup Shot', 'Floating Jump shot',
       'Hook Bank Shot', 'Hook Shot', 'Jump Bank Shot', 'Jump Shot',
       'Layup Shot', 'Pullup Jump shot', 'Putback Dunk Shot',
       'Putback Layup Shot', 'Reverse Dunk Shot', 'Reverse Layup Shot',
       'Running Alley Oop Dunk Shot', 'Running Alley Oop Layup Shot',
       'Running Dunk Shot', 'Running Finger Roll Layup Shot',
       'Running Jump Shot', 'Running Layup Shot', 'Running Pull-Up Jump Shot',
       'Running Reverse Dunk Shot', 'Running Reverse Layup Shot',
       'Step Back Bank Jump Shot', 'Step Back Jump shot', 'Tip Dunk Shot',
       'Tip Layup Shot', 'Turnaround Bank Hook Shot',
       'Turnaround Fadeaway Bank Jump Shot', 'Turnaround Fadeaway shot',
       'Turnaround Hook Shot', 'Turnaround Jump Shot', '2PT Field Goal',
       '3PT Field Goal']

In [16]:
agg = shots17_dummy[keep_cols].groupby('PLAYER_NAME').agg('mean')

## Create new features for GROUPED action types

### Groups

In [5]:
group1 = ['Alley Oop Dunk Shot',
       'Alley Oop Layup shot', 'Running Alley Oop Dunk Shot', 'Running Alley Oop Layup Shot']

group2 = ['Cutting Dunk Shot', 'Driving Dunk Shot', 'Driving Reverse Dunk Shot', 'Dunk Shot', 
          'Reverse Dunk Shot', 'Running Dunk Shot', 
         'Running Reverse Dunk Shot']

group3 = ['Running Finger Roll Layup Shot', 'Running Layup Shot','Cutting Finger Roll Layup Shot', 'Cutting Layup Shot', 
           'Driving Finger Roll Layup Shot', 'Driving Layup Shot', 
          'Driving Reverse Layup Shot', 'Finger Roll Layup Shot', 'Layup Shot', 'Reverse Layup Shot',
         'Running Reverse Layup Shot']

group4 = ['Driving Floating Bank Jump Shot',
       'Driving Floating Jump Shot', 'Floating Jump shot']

group5 = ['Driving Bank Hook Shot','Driving Hook Shot', 'Hook Bank Shot', 'Hook Shot', 'Turnaround Bank Hook Shot',
         'Turnaround Hook Shot']

group6 = ['Pullup Jump shot', 'Running Pull-Up Jump Shot', 'Step Back Bank Jump Shot',
         'Step Back Jump shot', 'Turnaround Fadeaway shot', 'Turnaround Fadeaway Bank Jump Shot', 
          'Turnaround Jump Shot'] 

group7 = ['Jump Bank Shot', 'Jump Shot', 'Fadeaway Jump Shot', 'Running Jump Shot']

group8 = ['Putback Dunk Shot', 'Putback Layup Shot', 'Tip Dunk Shot', 'Tip Layup Shot']

### Apply function below to action type column to create customized shot groups

In [6]:
def map_groups(action):
    if action in group1:
        return 'Alley-Oop'
    elif action in group2:
        return 'Dunk'
    elif action in group3:
        return 'Layup'
    elif action in group4:
        return 'Floater'
    elif action in group5:
        return 'Hook Shot'
    elif action in group6:
        return 'Created Jumper'
    elif action in group7:
        return 'Jump Shot'
    elif action in group8:
        return 'Rebound Shot'

In [78]:
shots17['shot_group'] = shots17['ACTION_TYPE'].map(lambda x: map_groups(x))

/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
shots17[['ACTION_TYPE', 'shot_group']]

,ACTION_TYPE,shot_group
0,Layup Shot,Layup
1,Tip Layup Shot,Rebound Shot
2,Driving Layup Shot,Layup
3,Cutting Dunk Shot,Dunk
4,Jump Shot,Jump Shot
...,...,...
472862,Jump Shot,Jump Shot
472863,Running Reverse Dunk Shot,Dunk
472864,Pullup Jump shot,Created Jumper
472865,Jump Shot,Jump Shot


In [80]:
shots17[shots17['shot_group'].isna()==True]['ACTION_TYPE']

Series([], Name: ACTION_TYPE, dtype: object)

### Get dummies for new shot groups

In [81]:
shots17_custom = pd.get_dummies(shots17, columns=['shot_group','SHOT_TYPE'], prefix=['',''], prefix_sep=['',''])
shots17_custom.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_ZONE_BASIC',
       'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM',
       'Alley-Oop', 'Created Jumper', 'Dunk', 'Floater', 'Hook Shot',
       'Jump Shot', 'Layup', 'Rebound Shot', '2PT Field Goal',
       '3PT Field Goal'],
      dtype='object')

### Calculate percentage of field goals attempts for each new group, for each player 

In [82]:
keep_cols2 =['PLAYER_ID', 'PLAYER_NAME', 'SHOT_DISTANCE', 'SHOT_MADE_FLAG',  'Alley-Oop', 
             'Created Jumper', 'Dunk', 'Floater', 'Hook Shot',
             'Jump Shot', 'Layup', 'Rebound Shot','2PT Field Goal',
             '3PT Field Goal']

agg2 = shots17_custom[keep_cols2].groupby('PLAYER_NAME').agg('mean')


### Calculate average distance of each type of new shot attempt by player, and field goal percentage for each shot type

In [84]:
grouped2 = shots17.groupby(by = ['PLAYER_NAME', 'shot_group']).agg('mean')
grouped2.head(50)

GAME_EVENT_ID  PLAYER_ID       TEAM_ID  \
PLAYER_NAME    shot_group                                               
Aaron Brooks   Created Jumper     377.133333   201166.0  1.610613e+09   
               Floater            338.142857   201166.0  1.610613e+09   
               Hook Shot          402.000000   201166.0  1.610613e+09   
               Jump Shot          479.965517   201166.0  1.610613e+09   
               Layup              450.125000   201166.0  1.610613e+09   
Aaron Gordon   Alley-Oop          220.638889   203932.0  1.610613e+09   
               Created Jumper     287.232955   203932.0  1.610613e+09   
               Dunk               270.012658   203932.0  1.610613e+09   
               Floater            307.176471   203932.0  1.610613e+09   
               Hook Shot          226.714286   203932.0  1.610613e+09   
               Jump Shot          285.562176   203932.0  1.610613e+09   
               Layup              283.259542   203932.0  1.610613e+09   
               Rebound Shot       252.515152   203932.0  1.610613e+09   
Aaron Harrison Created Jumper     295.363636  1626151.0  1.610613e+09   
               Dunk               469.000000  1626151.0  1.610613e+09   
               Floater            313.666667  1626151.0  1.610613e+09   
               Jump Shot          342.000000  1626151.0  1.610613e+09   
               Layup              310.833333  1626151.0  1.610613e+09   
               Rebound Shot       568.000000  1626151.0  1.610613e+09   
Aaron Jackson  Created Jumper     182.000000  1628935.0  1.610613e+09   
               Floater            469.000000  1628935.0  1.610613e+09   
               Jump Shot          422.333333  1628935.0  1.610613e+09   
               Layup              237.000000  1628935.0  1.610613e+09   
Abdel Nader    Alley-Oop          291.000000  1627846.0  1.610613e+09   
               Created Jumper     292.555556  1627846.0  1.610613e+09   
               Dunk               377.375000  1627846.0  1.610613e+09   
               Floater            352.500000  1627846.0  1.610613e+09   
               Hook Shot          181.000000  1627846.0  1.610613e+09   
               Jump Shot          382.611111  1627846.0  1.610613e+09   
               Layup              354.750000  1627846.0  1.610613e+09   
               Rebound Shot       416.166667  1627846.0  1.610613e+09   
Adreian Payne  Created Jumper     653.000000   203940.0  1.610613e+09   
               Hook Shot          189.000000   203940.0  1.610613e+09   
               Jump Shot          522.285714   203940.0  1.610613e+09   
               Rebound Shot       114.000000   203940.0  1.610613e+09   
Al Horford     Alley-Oop          358.047619   201143.0  1.610613e+09   
               Created Jumper     260.594059   201143.0  1.610613e+09   
               Dunk               245.193548   201143.0  1.610613e+09   
               Floater            291.941176   201143.0  1.610613e+09   
               Hook Shot          262.279661   201143.0  1.610613e+09   
               Jump Shot          322.826866   201143.0  1.610613e+09   
               Layup              304.991071   201143.0  1.610613e+09   
               Rebound Shot       389.722222   201143.0  1.610613e+09   
Al Jefferson   Created Jumper     308.387097     2744.0  1.610613e+09   
               Dunk               400.333333     2744.0  1.610613e+09   
               Floater            230.428571     2744.0  1.610613e+09   
               Hook Shot          307.958333     2744.0  1.610613e+09   
               Jump Shot          300.565789     2744.0  1.610613e+09   
               Layup              298.446809     2744.0  1.610613e+09   
               Rebound Shot       360.428571     2744.0  1.610613e+09   

                                 PERIOD  MINUTES_REMAINING  SECONDS_REMAINING  \
PLAYER_NAME    shot_group                                                       
Aaron Brooks   Created Jumper  3.000000           6.133333          36.9

### Get rid of miscellaneous columns and unstack dataframe so each stat becomes a column

In [89]:
formatted = grouped2[['SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG']].unstack()
formatted.columns

MultiIndex([( 'SHOT_DISTANCE',      'Alley-Oop'),
            ( 'SHOT_DISTANCE', 'Created Jumper'),
            ( 'SHOT_DISTANCE',           'Dunk'),
            ( 'SHOT_DISTANCE',        'Floater'),
            ( 'SHOT_DISTANCE',      'Hook Shot'),
            ( 'SHOT_DISTANCE',      'Jump Shot'),
            ( 'SHOT_DISTANCE',          'Layup'),
            ( 'SHOT_DISTANCE',   'Rebound Shot'),
            (         'LOC_X',      'Alley-Oop'),
            (         'LOC_X', 'Created Jumper'),
            (         'LOC_X',           'Dunk'),
            (         'LOC_X',        'Floater'),
            (         'LOC_X',      'Hook Shot'),
            (         'LOC_X',      'Jump Shot'),
            (         'LOC_X',          'Layup'),
            (         'LOC_X',   'Rebound Shot'),
            (         'LOC_Y',      'Alley-Oop'),
            (         'LOC_Y', 'Created Jumper'),
            (         'LOC_Y',           'Dunk'),
            (         'LOC_Y',        'Floater'),


### Remove multilevel column names

In [93]:
columns = []
categories = sorted(list(shots17.shot_group.unique()))
beginnings = ['avg_distance', 'avg_X_loc', 'avg_Y_loc', 'field_goal_percentage']
for start in beginnings:
    for category in categories:
        columns.append(start + '_' + category)

In [94]:
columns

['avg_distance_Alley-Oop',
 'avg_distance_Created Jumper',
 'avg_distance_Dunk',
 'avg_distance_Floater',
 'avg_distance_Hook Shot',
 'avg_distance_Jump Shot',
 'avg_distance_Layup',
 'avg_distance_Rebound Shot',
 'avg_X_loc_Alley-Oop',
 'avg_X_loc_Created Jumper',
 'avg_X_loc_Dunk',
 'avg_X_loc_Floater',
 'avg_X_loc_Hook Shot',
 'avg_X_loc_Jump Shot',
 'avg_X_loc_Layup',
 'avg_X_loc_Rebound Shot',
 'avg_Y_loc_Alley-Oop',
 'avg_Y_loc_Created Jumper',
 'avg_Y_loc_Dunk',
 'avg_Y_loc_Floater',
 'avg_Y_loc_Hook Shot',
 'avg_Y_loc_Jump Shot',
 'avg_Y_loc_Layup',
 'avg_Y_loc_Rebound Shot',
 'field_goal_percentage_Alley-Oop',
 'field_goal_percentage_Created Jumper',
 'field_goal_percentage_Dunk',
 'field_goal_percentage_Floater',
 'field_goal_percentage_Hook Shot',
 'field_goal_percentage_Jump Shot',
 'field_goal_percentage_Layup',
 'field_goal_percentage_Rebound Shot']

In [96]:
formatted.columns = columns
formatted.head()

,avg_distance_Alley-Oop,avg_distance_Created Jumper,avg_distance_Dunk,avg_distance_Floater,avg_distance_Hook Shot,avg_distance_Jump Shot,avg_distance_Layup,avg_distance_Rebound Shot,avg_X_loc_Alley-Oop,avg_X_loc_Created Jumper,...,avg_Y_loc_Layup,avg_Y_loc_Rebound Shot,field_goal_percentage_Alley-Oop,field_goal_percentage_Created Jumper,field_goal_percentage_Dunk,field_goal_percentage_Floater,field_goal_percentage_Hook Shot,field_goal_percentage_Jump Shot,field_goal_percentage_Layup,field_goal_percentage_Rebound Shot
PLAYER_NAME,,,,,,,,,,,,,,,,,,,,,
Aaron Brooks,NaN,20.533333,NaN,9.142857,7.000000,26.931034,1.750000,NaN,NaN,19.800000,...,17.375000,NaN,NaN,0.466667,NaN,0.142857,1.0,0.379310,0.437500,NaN
Aaron Gordon,1.0,17.982955,1.189873,7.764706,4.571429,21.891192,2.076336,0.363636,-0.138889,-19.409091,...,21.183206,-0.545455,0.861111,0.363636,0.848101,0.176471,0.0,0.318653,0.473282,0.757576
Aaron Harrison,NaN,16.909091,1.000000,8.833333,NaN,24.045455,1.166667,1.000000,NaN,46.727273,...,13.166667,4.000000,NaN,0.363636,1.000000,0.166667,NaN,0.204545,0.500000,1.000000
Aaron Jackson,NaN,24.000000,NaN,8.000000,NaN,24.666667,2.750000,NaN,NaN,-147.000000,...,26.250000,NaN,NaN,1.000000,NaN,1.000000,NaN,0.000000,0.250000,NaN
Abdel Nader,1.0,16.555556,1.375000,6.625000,9.000000,23.083333,2.909091,0.500000,-5.000000,-31.666667,...,24.022727,0.833333,0.000000,0.222222,0.750000,0.375000,0.0,0.319444,0.295455,0.500000


### Rename columns in agg2 dataframe for merge with above dataframe

In [104]:
agg2.head()

,PLAYER_ID,SHOT_DISTANCE,SHOT_MADE_FLAG,Alley-Oop,Created Jumper,Dunk,Floater,Hook Shot,Jump Shot,Layup,Rebound Shot,2PT Field Goal,3PT Field Goal
PLAYER_NAME,,,,,,,,,,,,,
Aaron Brooks,201166.0,17.318841,0.405797,0.000000,0.217391,0.000000,0.101449,0.028986,0.420290,0.231884,0.000000,0.550725,0.449275
Aaron Gordon,203932.0,14.095954,0.433526,0.041618,0.203468,0.091329,0.019653,0.008092,0.446243,0.151445,0.038150,0.604624,0.395376
Aaron Harrison,1626151.0,18.927536,0.275362,0.000000,0.159420,0.014493,0.086957,0.000000,0.637681,0.086957,0.014493,0.376812,0.623188
Aaron Jackson,1628935.0,13.000000,0.333333,0.000000,0.111111,0.000000,0.111111,0.000000,0.333333,0.444444,0.000000,0.555556,0.444444
Abdel Nader,1627846.0,13.530201,0.335570,0.006711,0.060403,0.053691,0.053691,0.006711,0.483221,0.295302,0.040268,0.563758,0.436242


In [105]:
agg_columns = ['player_id', 'avg_shot_distance_all', 'field_goal_percentage_all', 'percent_shots_Alley_Oop',
              'percent_shots_Created_Jumper', 'percent_shots_Dunk', 'percent_shots_Floater', 'percent_shots_Hook_Shot',
              'percent_shots_Jump_Shot', 'percent_shots_Layup', 'percent_shots_Rebound_Shot', 'percent_shots_2PTA', 'percent_shots_3PTA']
agg2.columns = agg_columns
agg2.head()

,player_id,avg_shot_distance_all,field_goal_percentage_all,percent_shots_Alley_Oop,percent_shots_Created_Jumper,percent_shots_Dunk,percent_shots_Floater,percent_shots_Hook_Shot,percent_shots_Jump_Shot,percent_shots_Layup,percent_shots_Rebound_Shot,percent_shots_2PTA,percent_shots_3PTA
PLAYER_NAME,,,,,,,,,,,,,
Aaron Brooks,201166.0,17.318841,0.405797,0.000000,0.217391,0.000000,0.101449,0.028986,0.420290,0.231884,0.000000,0.550725,0.449275
Aaron Gordon,203932.0,14.095954,0.433526,0.041618,0.203468,0.091329,0.019653,0.008092,0.446243,0.151445,0.038150,0.604624,0.395376
Aaron Harrison,1626151.0,18.927536,0.275362,0.000000,0.159420,0.014493,0.086957,0.000000,0.637681,0.086957,0.014493,0.376812,0.623188
Aaron Jackson,1628935.0,13.000000,0.333333,0.000000,0.111111,0.000000,0.111111,0.000000,0.333333,0.444444,0.000000,0.555556,0.444444
Abdel Nader,1627846.0,13.530201,0.335570,0.006711,0.060403,0.053691,0.053691,0.006711,0.483221,0.295302,0.040268,0.563758,0.436242


### Merge dataframes

In [114]:
combined = pd.merge(left=agg2, right=formatted, left_on=agg2.index, right_on=formatted.index, left_index=True)
combined.head()

,key_0,player_id,avg_shot_distance_all,field_goal_percentage_all,percent_shots_Alley_Oop,percent_shots_Created_Jumper,percent_shots_Dunk,percent_shots_Floater,percent_shots_Hook_Shot,percent_shots_Jump_Shot,...,avg_Y_loc_Layup,avg_Y_loc_Rebound Shot,field_goal_percentage_Alley-Oop,field_goal_percentage_Created Jumper,field_goal_percentage_Dunk,field_goal_percentage_Floater,field_goal_percentage_Hook Shot,field_goal_percentage_Jump Shot,field_goal_percentage_Layup,field_goal_percentage_Rebound Shot
PLAYER_NAME,,,,,,,,,,,,,,,,,,,,,
Aaron Brooks,Aaron Brooks,201166.0,17.318841,0.405797,0.000000,0.217391,0.000000,0.101449,0.028986,0.420290,...,17.375000,NaN,NaN,0.466667,NaN,0.142857,1.0,0.379310,0.437500,NaN
Aaron Gordon,Aaron Gordon,203932.0,14.095954,0.433526,0.041618,0.203468,0.091329,0.019653,0.008092,0.446243,...,21.183206,-0.545455,0.861111,0.363636,0.848101,0.176471,0.0,0.318653,0.473282,0.757576
Aaron Harrison,Aaron Harrison,1626151.0,18.927536,0.275362,0.000000,0.159420,0.014493,0.086957,0.000000,0.637681,...,13.166667,4.000000,NaN,0.363636,1.000000,0.166667,NaN,0.204545,0.500000,1.000000
Aaron Jackson,Aaron Jackson,1628935.0,13.000000,0.333333,0.000000,0.111111,0.000000,0.111111,0.000000,0.333333,...,26.250000,NaN,NaN,1.000000,NaN,1.000000,NaN,0.000000,0.250000,NaN
Abdel Nader,Abdel Nader,1627846.0,13.530201,0.335570,0.006711,0.060403,0.053691,0.053691,0.006711,0.483221,...,24.022727,0.833333,0.000000,0.222222,0.750000,0.375000,0.0,0.319444,0.295455,0.500000


## Calculate field goal attempts per game

In [120]:
shots_per = shots17.groupby('PLAYER_NAME')['SHOT_ATTEMPTED_FLAG', 'GAME_ID'].agg(['sum',pd.Series.nunique])

In [121]:
shots_per.columns = ['total_shots', 'xx', 'xxx', 'games_played']
shots_per['attempts_per_game'] = shots_per['total_shots'] / shots_per['games_played']
shots_per.head()

,total_shots,xx,xxx,games_played,attempts_per_game
PLAYER_NAME,,,,,
Aaron Brooks,69,1,0021700052002170005200217000520021700056002170...,23,3.000000
Aaron Gordon,865,1,0021700005002170000500217000050021700005002170...,58,14.913793
Aaron Harrison,69,1,0021701076002170107600217010760021701076002170...,9,7.666667
Aaron Jackson,9,1,0021701230002170123000217012300021701230002170...,1,9.000000
Abdel Nader,149,1,0021700007002170000700217000070021700007002170...,38,3.921053


In [122]:
shots_per.sort_values(by='attempts_per_game', ascending=False)

,total_shots,xx,xxx,games_played,attempts_per_game
PLAYER_NAME,,,,,
Russell Westbrook,1687,1,0021700015002170001500217000150021700015002170...,80,21.087500
James Harden,1449,1,0021700002002170000200217000020021700002002170...,72,20.125000
Devin Booker,1054,1,0021700012002170001200217000120021700012002170...,54,19.518519
Anthony Davis,1462,1,0021700008002170000800217000080021700008002170...,75,19.493333
Damian Lillard,1415,1,0021700012002170001200217000120021700012002170...,73,19.383562
...,...,...,...,...,...
Erik McCree,2,1,00217007930021701013,2,1.000000
Edmond Sumner,1,1,0021701213,1,1.000000
DaQuan Jeffries,1,1,0021900217,1,1.000000


### Merge with other info

#### Merge with ungrouped action types

In [20]:
merged = pd.merge(left=agg, right=shots_per['attempts_per_game'], left_on=agg.index, right_on=shots_per.index)

In [21]:
merged.head()

,key_0,PLAYER_ID,SHOT_DISTANCE,SHOT_MADE_FLAG,Alley Oop Dunk Shot,Alley Oop Layup shot,Cutting Dunk Shot,Cutting Finger Roll Layup Shot,Cutting Layup Shot,Driving Bank Hook Shot,...,Tip Dunk Shot,Tip Layup Shot,Turnaround Bank Hook Shot,Turnaround Fadeaway Bank Jump Shot,Turnaround Fadeaway shot,Turnaround Hook Shot,Turnaround Jump Shot,2PT Field Goal,3PT Field Goal,attempts_per_game
0,Aaron Brooks,201166.0,17.318841,0.405797,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.014493,0.000000,0.000000,0.00000,0.550725,0.449275,3.000000
1,Aaron Gordon,203932.0,14.095954,0.433526,0.019653,0.012717,0.011561,0.001156,0.010405,0.000000,...,0.002312,0.023121,0.0,0.003468,0.018497,0.000000,0.02659,0.604624,0.395376,14.913793
2,Aaron Harrison,1626151.0,18.927536,0.275362,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.376812,0.623188,7.666667
3,Aaron Jackson,1628935.0,13.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.555556,0.444444,9.000000
4,Abdel Nader,1627846.0,13.530201,0.335570,0.000000,0.006711,0.000000,0.000000,0.026846,0.006711,...,0.000000,0.020134,0.0,0.006711,0.013423,0.006711,0.00000,0.563758,0.436242,3.921053


In [23]:
merged.to_csv('initial_shot_chart_for_cluster.csv')

#### Merge with grouped Action Types

In [124]:
merged2 = pd.merge(left=combined.drop('key_0', axis=1), right=shots_per['attempts_per_game'], left_on=combined.index, right_on=shots_per.index)
merged2.head()


,key_0,player_id,avg_shot_distance_all,field_goal_percentage_all,percent_shots_Alley_Oop,percent_shots_Created_Jumper,percent_shots_Dunk,percent_shots_Floater,percent_shots_Hook_Shot,percent_shots_Jump_Shot,...,avg_Y_loc_Rebound Shot,field_goal_percentage_Alley-Oop,field_goal_percentage_Created Jumper,field_goal_percentage_Dunk,field_goal_percentage_Floater,field_goal_percentage_Hook Shot,field_goal_percentage_Jump Shot,field_goal_percentage_Layup,field_goal_percentage_Rebound Shot,attempts_per_game
0,Aaron Brooks,201166.0,17.318841,0.405797,0.000000,0.217391,0.000000,0.101449,0.028986,0.420290,...,NaN,NaN,0.466667,NaN,0.142857,1.0,0.379310,0.437500,NaN,3.000000
1,Aaron Gordon,203932.0,14.095954,0.433526,0.041618,0.203468,0.091329,0.019653,0.008092,0.446243,...,-0.545455,0.861111,0.363636,0.848101,0.176471,0.0,0.318653,0.473282,0.757576,14.913793
2,Aaron Harrison,1626151.0,18.927536,0.275362,0.000000,0.159420,0.014493,0.086957,0.000000,0.637681,...,4.000000,NaN,0.363636,1.000000,0.166667,NaN,0.204545,0.500000,1.000000,7.666667
3,Aaron Jackson,1628935.0,13.000000,0.333333,0.000000,0.111111,0.000000,0.111111,0.000000,0.333333,...,NaN,NaN,1.000000,NaN,1.000000,NaN,0.000000,0.250000,NaN,9.000000
4,Abdel Nader,1627846.0,13.530201,0.335570,0.006711,0.060403,0.053691,0.053691,0.006711,0.483221,...,0.833333,0.000000,0.222222,0.750000,0.375000,0.0,0.319444,0.295455,0.500000,3.921053


In [125]:
merged2.to_csv('grouped_shot_chart_for_cluster.csv')

## Function to re-run calculations for any season

In [7]:
## 2019 year should be '00219'
def create_grouped_df(year):
    stmt = "SELECT * from nba.shot_chart"
    cursor.execute(stmt)
    scores = cursor.fetchall()

    field_names = [i[0] for i in cursor.description]
    shots = pd.DataFrame(scores)
    shots.columns = field_names
    
    yr = shots[shots.GAME_ID.str.contains(year)]
    
    yr['shot_group'] = yr['ACTION_TYPE'].map(lambda x: map_groups(x))
    
    yr_custom = pd.get_dummies(yr, columns=['shot_group','SHOT_TYPE'], prefix=['',''], prefix_sep=['',''])
    
    keep_cols2 =['PLAYER_ID', 'PLAYER_NAME', 'SHOT_DISTANCE', 'SHOT_MADE_FLAG',  'Alley-Oop', 
             'Created Jumper', 'Dunk', 'Floater', 'Hook Shot',
             'Jump Shot', 'Layup', 'Rebound Shot','2PT Field Goal',
             '3PT Field Goal']

    agg2 = yr_custom[keep_cols2].groupby('PLAYER_NAME').agg('mean')
    
    grouped2 = yr.groupby(by = ['PLAYER_NAME', 'shot_group']).agg('mean')

    formatted = grouped2[['SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_MADE_FLAG']].unstack()
    
    columns = []
    categories = sorted(list(yr.shot_group.unique()))
    beginnings = ['avg_distance', 'avg_X_loc', 'avg_Y_loc', 'field_goal_percentage']
    for start in beginnings:
        for category in categories:
            columns.append(start + '_' + category)

    formatted.columns = columns

    agg_columns = ['player_id', 'avg_shot_distance_all', 'field_goal_percentage_all', 'percent_shots_Alley_Oop',
              'percent_shots_Created_Jumper', 'percent_shots_Dunk', 'percent_shots_Floater', 'percent_shots_Hook_Shot',
              'percent_shots_Jump_Shot', 'percent_shots_Layup', 'percent_shots_Rebound_Shot', 'percent_shots_2PTA', 'percent_shots_3PTA']
    
    agg2.columns = agg_columns
    
    combined = pd.merge(left=agg2, right=formatted, left_on=agg2.index, right_on=formatted.index, left_index=True)
    
    shots_per = yr.groupby('PLAYER_NAME')['SHOT_ATTEMPTED_FLAG', 'GAME_ID'].agg(['sum',pd.Series.nunique])
    
    shots_per.columns = ['total_shots', 'xx', 'xxx', 'games_played']
    shots_per['attempts_per_game'] = shots_per['total_shots'] / shots_per['games_played']
    
    merged2 = pd.merge(left=combined.drop('key_0', axis=1), right=shots_per['attempts_per_game'], left_on=combined.index, right_on=shots_per.index)

    return merged2


In [14]:
grouped_2018 = create_grouped_df('00218')

/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [16]:
grouped_2018.to_csv('custom_shot_groups_for_custering_2018.csv')

In [19]:
grouped_2019 = create_grouped_df('00219')

/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [21]:
grouped_2019.to_csv('custom_shot_groups_for_custering_2019.csv')

In [8]:
group_2017 = create_grouped_df('00217')
group_2018 = create_grouped_df('00218')
group_2019 = create_grouped_df('00219')

/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/marshall132/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [9]:
groups = [group_2017, group_2018, group_2019]
year = ['2017', '2018', '2019']
for idx, group in enumerate(groups):
    group.to_csv('group_{}.csv'.format(year[idx]))